# 异步和性能

## 异步

In [ ]:
{
    //伪代码
    var eventLoop = [];
    var event;
    
    while(true){
        if(eventLoop.length > 0){
            event = eventLoop.shift();
            
            try{
                event();
            }catch(err){
                reportError(err);
            }
        }
    }
}

## 回调

## promise

In [1]:
{
    var p = new Promise(function(resolve,reject){
        foo.bar();
        resolve(42);
    })
    
    p.then(function(){
        console.log(`never`);
    },function(err){
        console.log(`err is ${err}`);
    })
}

err is ReferenceError: foo is not defined


In [3]:
{
    var p = new Promise(function(resolve,reject){
        resolve(42);
    });
    
    p.then(function(){
        foo.bar();
        console.log(`this is an error`);
    },function(err){
        console.log(`err is ${err}`);
    }).then(function(suc){
        console.log(`then suc is ${suc}`)
    },function(err){
        console.log(`then err is ${err}`);
    })
}

then err is ReferenceError: foo is not defined


In [ ]:
{
    Promise.all([])//等全部完成
    Promise.race([])//等待第一个完成
}

`then` 接受一个或两个参数：1 用于完成回调，2 用于拒绝回调
`catch` 接受一个拒绝回调 相当于`then(null,cb)`

## 生成器

In [6]:
{
    var x = 1;
    
    function *foo(){
        x++;
        yield;
        console.log(`iterator x is ${x}`);
    }
    
    function bar(){
        x++;
    }
    
    var it = foo();
    console.log(`1x is ${x}`);
    bar();
    console.log(`2x is ${x}`);
    it.next();
    console.log(`3x is ${x}`);
    it.next();
    console.log(`4x is ${x}`);
}

1x is 1
2x is 2
3x is 3
iterator x is 3
4x is 3


In [5]:
{
    function *foo(x){
        let y = x *(yield);
        return y;
    }
    
    let it = foo(6);
    console.log(`it.next() is ${JSON.stringify(it.next())}`);
    
    let res = it.next(7);
    console.log(`res is ${JSON.stringify(res)}`);
}

it.next() is {"done":false}
res is {"value":42,"done":true}


In [8]:
{
    function *foo(x){
        let y = x *(yield "hello");
        return y;
    }
    
    let it = foo(6);
    let res = it.next();
    console.log(`${JSON.stringify(res)}`);
    res = it.next(7);
    console.log(`${JSON.stringify(res)}`);
}

{"value":"hello","done":false}
{"value":42,"done":true}


In [11]:
{
    let a = 1;
    let b = 2;
    
    function *foo(){
        a++;
        yield;
        b = b*a;
        a = (yield b) + 3;
    }
    
    function *bar(){
        b--;
        yield;
        a = (yield 8)+b;
        b = a*(yield 2);
    }
    
    function setp(gen){
        let it = gen();
        let last;
        
        return function(){
            last = it.next(last).value;
        }
    }
    
    let s1 = setp(foo);
    let s2 = setp(bar);
    
//     s1();
//     s1();
//     s1();
    
//     s2();
//     s2();
//     s2();
//     s2();
    
    s2();
    s2();
    s1();
    s2();
    
    s1();
    
    s1();
    s2();
    
    console.log(a,b);
}

12 18


In [13]:
{
    let gimmeSomething = (function(){
        let nextVal;
        
        return function(){
            if(nextVal === undefined){
                nextVal = 1;
            }else{
                nextVal = (3*nextVal)+6;
            }
            return nextVal;
        }
    })();
    
    console.log(gimmeSomething());
    console.log(gimmeSomething());
    console.log(gimmeSomething());
    console.log(gimmeSomething());
}

1
9
33
105


In [14]:
{
    let something = (function(){
        var nextVal;
        
        return {
            [Symbol.iterator]:function(){return this;},
            next:function(){
                if(nextVal===undefined){
                    nextVal = 1;
                }else{
                    nextVal = (3*nextVal)+6;
                }
                
                return {done:false,value:nextVal};
            }
        }
    })();
    
    console.log(something.next().value);
    console.log(something.next().value);
    console.log(something.next().value);
    console.log(something.next().value);
}

1
9
33
105


In [16]:
{
    function *something(){
        try{
            var nextVal;
            while(true){
                if(nextVal === undefined){
                    nextVal = 1;
                }else{
                    nextVal = (3*nextVal)+6;
                }

                yield nextVal;
            }
        }
        finally{
            console.log(`cleaning up!`);
        }        
    }
    var it = something();
    for(let v of it){
        console.log(v);
        if(v>500){
            console.log(
                it.return('hello').value
            )
        }
    }
}

1
9
33
105
321
969
cleaning up!
hello


In [20]:
{
    function foo(flag){
        setTimeout(function(){
            if(flag){
                it.throw(flag);
            }else{
                it.next('success')
            }
        },1000);
    }
    
    function *main(){
        try{
            let text = yield foo(false);
            console.log(`text is ${text}`);
        }catch(err){
            console.error(err);
        }
    }
    
    let it = main();
    it.next();
}

{ value: undefined, done: false }

text is success


In [21]:
{
    function *main(){
        let x = yield "hello world";
        console.log(x);
    }
    
    let it = main();
    
    it.next();
    
    try{
        it.throw('Oops');
    }catch(err){
        console.error(err);
    }
}

Oops


In [27]:
{
    function foo(flag){
        return new Promise(function(resolve,reject){
            setTimeout(function(){
                if(flag){
                    resolve('resolve');
                }else{
                    reject('reject');
                }                           
            },1000);
        })
    }
    
    function *main(){
        try{
            let text = yield foo(false/*or true*/);
            console.log(text);
        }
        catch(err){
            console.error(err);
        }
    }
    
    let it = main();
    let p = it.next().value;
    console.log(p);
    p.then(function(text){
        it.next(text);
    },function(text){
        it.throw(text);
    })
    
}

Promise { <pending> }


reject


In [ ]:
{
    function run(gen){
        var args = [].slice.call(arguments,1),it;
        
        it = gen.apply(this,args);
        
        return Promise.resolve().
            then(function handleNext(value){
                var next = it.next(value);
                
                return (function handleResult(next){
                    if(next.done){
                        return next.value;
                    }else{
                        return Promise.resolve(next.value).
                            then(handleNext,function handleErr(err){
                                return Promise.resolve(
                                    it.throw(err)
                                ).then(handleResult);
                        });
                    }
                })(next);
        })
    }
}

In [29]:
{
    function foo(flag){
        return new Promise(function(resolve,reject){
            setTimeout(function(){
                if(flag){
                    resolve('resolve');
                }else{
                    reject('reject');
                }                           
            },1000);
        })
    }
    
    async function main(){
        try{
            let text = await foo(false/*or true*/);
            console.log(text);
        }
        catch(err){
            console.error(err);
        }
    }
    
    main();
    
}

reject


`yield * __` yield 委托

In [30]:
{
    function *foo(){
        console.log(`*foo() staring`);
        yield 3;
        yield 4;
        console.log(`*foo() finished`);
    }
    function *bar(){
        yield 1;
        yield 2;
        yield *foo();
        yield 5;
    }
    
    let it = bar();
    console.log(`1.it.next() is ${JSON.stringify(it.next())}`);
    console.log(`2.it.next() is ${JSON.stringify(it.next())}`);
    console.log(`3.it.next() is ${JSON.stringify(it.next())}`);
    console.log(`4.it.next() is ${JSON.stringify(it.next())}`);
    console.log(`5.it.next() is ${JSON.stringify(it.next())}`);
}

1.it.next() is {"value":1,"done":false}
2.it.next() is {"value":2,"done":false}
*foo() staring
3.it.next() is {"value":3,"done":false}
4.it.next() is {"value":4,"done":false}
*foo() finished
5.it.next() is {"value":5,"done":false}


## 程序性能

### web worker

worker之间以及它们和主程序之间，不会共享任何作用域或资源。而是通过一个基本的事件消息机制相互联系。

web worker不能访问主程序的任何全局变量，也不能访问页面DOM或其他资源。

但是可以进行网络操作(`ajax`,`websockets`) 以及设定定时器。

web worker可以访问`navigator`,`location`,`JSON`,`applicationCache`.

还可以通过`importScripts(...)`加载额外的javascript脚本。

web worker 常用于一下方面
- 处理密集型数学计算
- 大数据集排序
- 数据处理 (压缩、音频分析、图像处理等)
- 高流量网络通信

In [ ]:
{
    var w1 = new Worker("webworker.js");
    
    w1.addEventListener('message',function(evt){
        console.log(`w1 ge message is ${evt}`);
    })
    
    w1.postMessage('go in');
}

### SIMD 单指令多数据

## 性能测试与调优

In [32]:
{
    var start = (new Date()).getTime();
    var end = (new Date()).getTime();
    console.log(`Duration:${end-start}`);
}

Duration:1
